In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

In [7]:
spark = SparkSession.builder.appName("StructuredStreamingTest").getOrCreate()

In [8]:
# value 라는 column 하나, 스트리밍하게 입력받는 데이터들이 각각 row 를 구성하는 DataFrame
# Warning 은 무시. 소켓으로부터 스트림을 읽는 것은 서비스 배포용이 아닌 학습용으로만 사용하라는 경고
lines = spark.readStream.format("socket").option("host", "localhost").option("port", "5000").load()
print(lines)

DataFrame[value: string]


21/09/22 12:02:19 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [9]:
# SparkSQL 에서 제공되는 함수를 사용해서 transformation
# lines 데이터프레임의 value 컬럼을 공백으로 split 하고, 각 원소들을 row 로 하는 word 컬럼 생성
words = lines.select(explode(split(lines.value, " ")).alias("word"))
print(words)

DataFrame[word: string]


In [10]:
# words 데이터프레임에서 word 로 집계한다음 각 원소들의 수를 센 결과 데이터프레임이 wordCounts
# 일반적인 테이블에서 집계하는 것과 동일하지만, 스파크가 지속적으로 소켓에서 데이터를 확인해서, incremental query 를 실행시킴
wordCounts = words.groupBy("word").count()
print(wordCounts)

DataFrame[word: string, count: bigint]


In [ ]:
# 결과테이블을 complete 모드로, 콘솔 싱크에 write
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

21/09/22 12:02:27 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9d92c647-2e8e-40cc-b12a-59ae56be071a. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
|   3|    1|
|   5|    1|
|   6|    1|
|1000|    1|
|   e|    1|
|   d|    1|
|   c|    2|
|   1|    1|
|   b|    2|
|   4|    1|
|   a|    2|
|   2|    1|
+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|     3|    1|
|    ef|    1|
|     5|    1|
|     6|    1|
|  1000|    1|
|     e|    1|
|vi=vmi|    1|
| alias|    1|
|     d|    2|
|     c|    2|
|     1|    1|
|     b|    2|
|     4|    1|
|     a|    2|
|      |    5|
|     2|    1|
+------+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|     3|    1|
|    ef|    1|
|     5|    1|
|     6|    1|
|  1000|    1|
|     e|    1|
|vi=vmi|    1|
| alias|    1|
|     d|    2|
|     c|    2|
|     1|    2|
|     b|    2|
|     4|    1|
|     a|    5|
|      |    5|
|     2|    1|
+------+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|     3|    1|
|    ef|    1|
|     5|    1|
|     6|    1|
|  1000|    1|
|     e|    1|
|vi=vmi|    1|
| alias|    1|
|     d|    2|
|     c|    2|
|     1|    2|
|     b|    2|
|     4|    1|
|     a|   12|
|      |    5|
|     2|    1|
+------+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|     3|    1|
|    ef|    1|
|     5|    1|
|     6|    1|
|  1000|    1|
|     e|    1|
|vi=vmi|    1|
| alias|    1|
|     d|    2|
|     c|    2|
|     1|    2|
|     b|    2|
|     4|    1|
|     a|   20|
|      |    5|
|     2|    1|
+------+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+------+-----+
|  word|count|
+------+-----+
|     3|    1|
|    ef|    1|
|     5|    1|
|     6|    1|
|  1000|    1|
|     e|    1|
|vi=vmi|    1|
| alias|    1|
|     d|    2|
|     c|    2|
|     1|    2|
|     b|    2|
|     4|    1|
|     a|   26|
|      |    6|
|     2|    1|
+------+-----+



21/09/22 12:18:08 WARN TextSocketMicroBatchStream: Stream closed by localhost:5000


# Programming Model

Structured Streaming 의 핵심 아이디어는, 계속해서 데이터가 append 되는 테이블을 통해 데이터 스트림을 처리하는 것.

이는 배치 프로세싱 모델과 유사함.

스파크는 Unbounded input table(스트림 데이터가 계속 추가되는) 위에서 동작하는 incremental query 로써 배치 프로세싱과 비슷한 스트림 처리를 가능하게 함.

데이터가 들어올때마다 unbounded table 의 뒷부분에 새로운 row 로써 추가됨.

테이블에 행한 쿼리는 결과테이블을 만들어내는데, 특정 trigger interval 마다 새로운 데이터들이 입력테이블에 추가되면서 결과테이블을 갱신함.


다른 스트리밍과의 중요한 차이점은, 많은 다른 스트리밍 시스템들은 사용자가 falut-tolerance 나 data consistency 를 위해서 보증해야할 부분들이 많지만, Structured는 새로운 데이터가 들어왔을 때 결과 테이블에 대한 갱신만 책임지면 된다는 점에서 부담감이 적다는 것. (```무슨 말?```)

- 데이터가 소스 로부터 정의된 스키마에 맞게 들어옴
- 이벤트 스트림이란 unbounded table 에 추가되는 행들
- 스트림으로부터 결과를 얻기 위해 unbounded table 에서 쿼리를 날림
- 동일한 쿼리를 테이블에 지속적으로 날려 이벤트의 처리 결과 테이블을 생성
- 결과를 싱크에 제공

따라서, 배치 데이터를 테이블로 받아와서 테이블에서 쿼리를 통해 데이터를 처리할 수 있다면, 이를 스트림처리에 그대로 가져와서 스트리밍 데이터를 넣어주는 방식으로 변환할 수 있음 -> 이건 스파크 sql 의 내용이므로 자세히 다루지 않음

테이블 변환시켜서 쿼리 날리는게 궁금하다면? [Spark SQL 빌트인 함수들](https://spark.apache.org/docs/latest/api/sql/index.html)

스파크sql 이 아닌 구조적 스트리밍의 관점에서 알아야 할 내용들에 대해서만 알아보자

---

## 스파크 세션

스파크쉘에서 돌릴 경우는 SparkContext 처럼 SparkSession 이 제공됨

In [ ]:
spark = SparkSession.builder.appName("StructuredStreamingTest").getOrCreate()

---

## 소스 (Source)
소스란 스트리밍 데이터 공급자를 나타내는 추상화된 개념. 그냥 데이터가 생성되는 곳

구조적 스트리밍은 외부의 스트리밍 소스에서 현재 오프셋을 요청하고, 자기가 이전에 마지막으로 처리한 오프셋과 비교하여 그 사이에 있는 배치 데이터를 가져와 처리.
- 신뢰할 수 있는 소스란? 스트리밍 소스를 동일한 순서로 다시 재생시킬 수 있어야 함(스트리밍 프로세스가 실패하더라도 커밋되지 않은 오프셋을 재생성해서 다시 처리할 수 있어야 함), ```replayability```

In [ ]:
lines = spark.readStream.format("socket").option("host", "localhost").option("port", "5000").load()

---

### format
- file : 파일시스템에 존재하는 파일을 읽어옴, 배치 기반 프로세스를 스트리밍 시스템으로 연결하기 위한 간단한 방법. 스파크 2.3.0 부터 csv, json, parquet, orc, text, textfile 등을 지원
- kafka : 카프카 토픽으로부터 데이터를 컨슘해오는 subscriber 가 됨
- socket : utf-8 로 인코딩된 텍스트 데이터 스트림을 제공하는 tcp 서버에 연결해서 데이터를 받아오는 클라이언트
- rate : 초당 몇개의 레코드를 생성해내게끔 설정해줄 수 있는 스트림 생성기

### load

load 를 통해 받아온 결과는 스트리밍되는 DataFrame

---

## 싱크 (Sink)
failure 에 대비하여, 스트림 처리를 다시 실행시키더라도 idempotent 하게 설계되었음

In [ ]:
query = wordCounts.writeStream.outputMode("complete").format("console").start()

### outputMode
- append : 이번 인터벌에 들어온 새로운 row 들만 처리, 수신한 스트림을 처리한 결과가 수정되지 않는 경우(proj, filtering, mapping 등의 선형 변환)
- complete : 결과 테이블을 싹 갱신해서 싱크에 write. 실질적으로는 낮은 카디널리티 기준에 따라 집계하는 경우에만 권장(row 가 적어야 함)
- update : 결과테이블 중 바뀌는 row들만 write (즉, 집계 등과 같이 결과테이블의 다른 row 에 영향을 주는 스트림처리가 아니라면 append 와 동일)

### format
- console : stdout 에 출력
- memory : 메모리에 테이블을 만들고 지속적인 갱신
- file : 파일시스템에 특정 형식으로 - csv, json, avro, text 등
- kafka : 카프카 토픽에 프로듀싱

### start
DataStreamWriter 가 실제로 수행될 수 있게 전체 Job을 스트리밍 연산으로 구체화시키고 내부적으로 스케줄링 프로세스를 시작시킴, 쿼리를 관리하는 StreamingQuery 객체를 반환.

---

# Handling Event-time and Late Data

이벤트가 생성된 타임라인의 관점에서 처리 로직을 적용하고 싶음 -> 각 기기들에서 timestamp 를 찍어서 데이터를 보냄

스트림이 하나의 row 를 이루기 때문에, 타임스탬프는 그 중 하나의 컬럼이 되는 것.

처리하는 시스템의 내부 시계가 아닌 생산 시스템의 관점에서 이벤트들의 타임라인을 해석해야 함 - 찍힌 타임스탬프를 기준으로

따라서 일반적으로 타임스탬프로 선언된 필드가 단조 증가하면서 타임라인이 증가 -> 이벤트가 늦게 도착할 수 있음

현재 설정된 타임라인보다 일정 시간 이상 차이나는 이벤트들을 폐기시키는 워터마크

---

# Window

---

# Join